In [1]:
!pip install easyocr  --user

In [4]:
!pip install numpy==1.24.2 --user

## import

In [5]:
from matplotlib import pyplot as plt
from imutils.perspective import four_point_transform
from imutils.contours import sort_contours
import imutils
from easyocr import Reader
import cv2
import requests
import numpy as np
from PIL import ImageFont, ImageDraw, Image

### 이미지 확인 function / scan 이미지 변환 function

In [6]:
def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)
 
    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []
 
            for i in range(len(img)):
                titles.append(title)
 
        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
 
            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
 
        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()
 
 
def make_scan_image(image, width, ksize=(5,5), min_threshold=75, max_threshold=200):
  image_list_title = []
  image_list = []
 
  image = imutils.resize(image, width=width)
  ratio = org_image.shape[1] / float(image.shape[1])
 
  # 이미지를 grayscale로 변환하고 blur를 적용
  # 모서리를 찾기위한 이미지 연산
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blurred = cv2.GaussianBlur(gray, ksize, 0)
  edged = cv2.Canny(blurred, min_threshold, max_threshold)
 
  image_list_title = ['gray', 'blurred', 'edged']
  image_list = [gray, blurred, edged]
 
  # contours를 찾아 크기순으로 정렬
  cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
 
  findCnt = None
 
  # 정렬된 contours를 반복문으로 수행하며 4개의 꼭지점을 갖는 도형을 검출
  for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # contours가 크기순으로 정렬되어 있기때문에 제일 첫번째 사각형을 영역으로 판단하고 break
    if len(approx) == 4:
      findCnt = approx
      break
 
 
  # 만약 추출한 윤곽이 없을 경우 오류
  if findCnt is None:
    raise Exception(("Could not find outline."))
 
 
  output = image.copy()
  cv2.drawContours(output, [findCnt], -1, (0, 255, 0), 2)
  
  image_list_title.append("Outline")
  image_list.append(output)
 
  # 원본 이미지에 찾은 윤곽을 기준으로 이미지를 보정
  transform_image = four_point_transform(org_image, findCnt.reshape(4, 2) * ratio)
 
  plt_imshow(image_list_title, image_list)
  plt_imshow("Transform", transform_image)
 
  return transform_image

한글 표기 function

In [7]:
def putText(cv_img, text, x, y, color=(0, 0, 0), font_size=22):
  # Colab이 아닌 Local에서 수행 시에는 gulim.ttc 를 사용하면 됩니다.
  # font = ImageFont.truetype("fonts/gulim.ttc", font_size)
  font = ImageFont.truetype('/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf', font_size)
  img = Image.fromarray(cv_img)
   
  draw = ImageDraw.Draw(img)
  draw.text((x, y), text, font=font, fill=color)
 
  cv_img = np.array(img)
  
  return cv_img

대비 조절 함수

In [39]:
def change_contrast(img, alpha=0.0):
    '''
    명암비(contrast)를 변환하는 함수
    [parameter]
        img: ndarray - 명암비를 조절할 대상 이미지
        alpha: float -  대비 조절 비율값. 기본: 0.0(원본) - 양수: 명암비를 높인다. 음수 - 명암비를 낮춘다. 
    [return]
        ndarray - 명암비가 변환된 이미지
    '''
    return np.clip((1.0 + alpha) * img - 128 * alpha, 0, 255).astype('uint8')


# Load Image

In [ ]:
## 0.이미지 불러오기
path = r"C:\Users\Playdata\Desktop\jjj_project\tesseract\images\test11.jpg"
image = cv2.imread(path)  

plt_imshow('image', image)

In [ ]:
image_ = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)  # grayscale
# image_ = cv2.medianBlur(image_, ksize=3)  # noise 제거
image_ = change_contrast(image_, 1)  # 대비 높임

plt_imshow('image proprecessing', image_)

In [47]:
### EasyOCR 초기화 (사용할 언어를 지정)
langs = ['ko', 'en']
reader = Reader(lang_list=langs)

results = reader.readtext(image_)  
# 추출 글자에 대해 각 4개의 꼭지점 좌표, 인식 글자, 확률을 리스트로 return

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [48]:
results

[([[323, 201], [391, 201], [391, 215], [323, 215]],
  '11nssi-',
  0.08492545189415235)]

In [37]:
# detail=0 : 인식된 글자만 추출 list 형태로 return
simple_results = reader.readtext(image, detail = 0)
simple_results

[]

### 문제점
웹캠 화질이 좋지 않아 ocr 성능이 떨어진다.
ip webcam을 연결해서 핸드폰 카메라로 실행

## IP webcam 연결

In [7]:
#### 한 장씩 얻어오는 코드 ####
import urllib.request
import cv2
import numpy as np
import time

# Replace the URL with your own IPwebcam shot.jpg IP:port
url='http://172.30.1.40:8080/shot.jpg'

while True:
    
    # Use urllib to get the image and convert into a cv2 usable format
    imgResp = urllib.request.urlopen(url)
    imgNp = np.array(bytearray(imgResp.read()), dtype=np.uint8)
    img = cv2.imdecode(imgNp,-1)
    # put the image on screen
    cv2.imshow('IPWebcam',img)

    #To give the processor some less stress
    #time.sleep(0.1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [3]:
#### 영상으로 불러오는 코드 ####
import urllib.request
import cv2
import numpy as np
import time

ip_address = "172.30.1.40"
port = "8080"

url = f"http://{ip_address}:{port}/video"

cap = cv2.VideoCapture(url)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    cv2.imshow("IP Webcam Video Stream", frame)

    # 'q' 키를 누르면 종료합니다
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(0.1)

cap.release()
cv2.destroyAllWindows()